In [30]:
# Standard libraries
import os
import json
import time

# Data handling
import pandas as pd
import numpy as np

# Prediction guard and language models
from predictionguard import PredictionGuard
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Torch and Skorch
import torch
from torch import nn
from torch.optim import Adam
from skorch import NeuralNetClassifier

# Transformers (for embeddings)
from transformers import BertTokenizer, BertModel

# Machine learning models and utilities
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Metrics and performance evaluation
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    auc,
    RocCurveDisplay,
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    f1_score
)

# Plotting
import matplotlib.pyplot as plt

# XGBoost
from xgboost import XGBClassifier

# Imbalanced-learn for oversampling
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Surpress Warnings
import warnings
warnings.filterwarnings("ignore")

In [31]:
# Grief Journey Map
JourneyMap = {
    "1. Immediate Grief, Shock, and Emotion": "Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.", 

    "2. Navigating Family Relationships": "Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.",

    "3. Learning to Process Grief": "Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.", 

    "4. Moments That Matter": "Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special moments (sports, school achievements, moments that matter).", 

    "5. Feeling Immersed, Connected & Seen": "Finding new purpose and goals to begin moving towards Positive Integration. Surviving Family Unit: Connected to a broader community; support system; not the only person/family experiencing loss.", 

    "6. New Growth & Purpose": "Healthy point in grief journey; feeling capable to help others and a desire to do so. Surviving Family Unit: Ready to give back to the TAPS community through mentorship programs, volunteering at charity drives & events, etc." 
    }

dict = {
    "Stage Number": [int(key.split('.')[0]) for key in JourneyMap.keys()],
    "Stage Name": [key.split('. ', 1)[1] for key in JourneyMap.keys()],
    "Description": JourneyMap.values()
}

JourneyMapDF = pd.DataFrame(dict)
JourneyMapDF.set_index("Stage Number", inplace=True)
JourneyMapDF.head()

,Stage Name,Description
Stage Number,,
1,"Immediate Grief, Shock, and Emotion","Overwhelmed, loss of purpose; shock and trauma..."
2,Navigating Family Relationships,Experiencing tension between individuals withi...
3,Learning to Process Grief,Experiencing grief and learning to process tho...
4,Moments That Matter,Renewed experience of grief around anniversari...
5,"Feeling Immersed, Connected & Seen",Finding new purpose and goals to begin moving ...


In [32]:
# Survey Questions
SurveyQuestions = [
    {
        "question": "Please share all of the reasons you chose to attend the TAPS [SEMINAR NAME]",
        "options": [
            "I had attended a previous Regional or Nationals seminar",
            "To connect with other survivors who share a similar loss",
            "To learn more about resources TAPS has to offer",
            "To learn new tools and information to help me with my grief",
            "To learn more about how to support my adult family members in their grief",
            "To learn more about how to support my child(ren) in their grief",
            "For my child(ren) to attend Good Grief Camp",
            "For my child(ren) to connect with a Military Mentor"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "How did you find out about the TAPS [SEMINAR NAME]?",
        "options": [
            "I found this event while searching for grief resources",
            "I attended a seminar last year and had already marked my calendar!",
            "TAPS invited me to this event via email",
            "My TAPS Survivor Care Team Member invited me",
            "My Peer Mentor or another survivor invited me",
            "Through the TAPS website",
            "Through the TAPS magazine",
            "Through a TAPS Social Media Page"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "This seminar helped me to feel socially connected",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar taught me new ways to cope with my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar helped me to better understand my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar has given me hope for the future",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to connect with TAPS virtually / at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to bring your child(ren) to connect with TAPS at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "I have a positive outlook toward life",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have short and/or long range goals",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel all alone",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can see possibilities in the midst of difficulties",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have faith that gives me comfort",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel scared about my future",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can recall happy/joyful times",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have deep inner strength",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I am able to give and receive care/love",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have a sense of direction",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I believe that each day has potential",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel my life has value and worth",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?",
        "options": "Write a free-response.",
        "type": "free_text"
    },
    {
        "question": "Please share any additional feedback you have regarding your TAPS Seminar experience.",
        "options": "Write a free-response.",
        "type": "free_text"
    }
]

In [33]:
# Training Data Import
data = pd.read_csv("examples.csv")

In [48]:
# Function to create prompt with stage-specific examples for free-text questions
def specific_prompt(stage_number, question_text, options_text, data):
    # Filter the data for the specific grief stage
    stage_data = data[data["label"] == stage_number]
    
    # Limit to a few examples to keep the prompt concise
    stage_examples = stage_data.sample(min(1, len(stage_data)))

    # Format examples for the prompt
    example_texts = "\n\n".join(
        [
            f"Example {i+1}:\n" +
            "\n".join([f"Q{idx+1}: {row[f'q{idx+1}']}" for idx in range(20)]) + 
            (f"\nFree Text Response 1: {row['q21']}" if pd.notnull(row['q21']) else "") +
            (f"\nFree Text Response 2: {row['q22']}" if pd.notnull(row['q22']) else "")
            for i, row in stage_examples.iterrows()
        ]
    )

    # Structure the prompt messages for the LLM to generate a free-text response
    messages = [
        {
            "role": "user",
            "content": f"""
            {example_texts}
            
            Current Question: {question_text}
            Options: {options_text}
            If the question is question 1-20 (multiple choice), respond only with 1 number corresponding to your answer. If the question is 21 or 22 (free-response), please write a free response answer. Do not provide any extra information, just answer the question!
            Response:
            """
        }
    ]

    return messages


In [60]:
# Revised DataFrame handling
responses_list = []
os.environ["PREDICTIONGUARD_API_KEY"] = "Oq62vYfSJRwjnFQcUnJy5PM3SRVejYtJCXWSxnfv"
client = PredictionGuard()
# Revised loop
for run_id in range(50):
    for stage, description in JourneyMap.items():
        stage_number = int(stage.split('.')[0])
        stagename = stage.split('.', 1)[1].replace(' ', '_')
        responses = {"Id": f"{run_id}_{stagename}", "label": stage_number}
        
        basemessage = [
            {
                "role": "system",
                "content": f"""
                You are a person who has experienced the loss of a loved one in the military or veteran community. You are currently at the '{stagename}' stage of grief: {description}. You are asked by the Tragedy Assistance Program for Survivors (TAPS) to fill out this survey based on your current stage of grief. Here are some example responses from others in this stage to guide your answer. 
                
                
                
                If the question is question 1-20 (multiple choice), respond only with 1 number corresponding to your answer. If the question is 21 or 22 (free-response), please write a free response answer IN ENGLISH. Do not provide any commentary, just answer the question!
                """
            }
        ]

        for question_index, question_data in enumerate(SurveyQuestions[:22]):
            question_text = question_data["question"]
            options_text = ""
            
            # Only set options_text if it's not a free text question
            if question_data["type"] != "free_text":
                options_text = "\n".join([f"{i}. {option}" for i, option in enumerate(question_data["options"])])
            
            messages = specific_prompt(
                stage_number=stage_number,
                question_text=question_text,
                options_text=options_text,
                data=data
            )
            basemessage.extend(messages)  # Flatten the list

            # Set max_tokens based on question index
            max_tokens = 2 if question_index < 20 else 150

            result = client.chat.completions.create(
                model="Hermes-3-Llama-3.1-70B",
                messages=basemessage[-5:],
                temperature=0.5,
                max_tokens=max_tokens
            )
            answer = result["choices"][0]["message"]["content"].strip()
            responses[f"q{question_index+1}"] = answer
            time.sleep(0.1)
            assistant_response = [
                {
                    "role": "assistant",
                    "content": f"{answer}"

                }
            ]
            basemessage.extend(assistant_response)

        # Append to list instead of DataFrame directly
        responses_list.append(responses)

# Convert list of responses to DataFrame at the end
responses_df = pd.DataFrame(responses_list)


In [61]:
responses_df

,Id,label,q1,q2,q3,q4,q5,q6,q7,q8,...,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22
0,"0__Immediate_Grief,_Shock,_and_Emotion",1,1,2,2,2,2,2,1,1,...,2,2,2,2,2,2,2,2,My favorite moment of the weekend was spending...,The TAPS Seminar was a valuable and meaningful...
1,0__Navigating_Family_Relationships,2,1,2,2,2,2,2,0,0,...,2,1,2,2,2,2,2,2,Mi momento favorito del fin de semana fue dura...,Mi experiencia en el seminario TAPS fue sumame...
2,0__Learning_to_Process_Grief,3,1,2,2,2,2,2,0,0,...,2,1,2,2,2,2,2,2,Free Text Response 1: My favorite moment of th...,Free Text Response 2: The TAPS Seminar was a t...
3,0__Moments_That_Matter,4,3,2,2,2,2,2,0,1,...,2,1,2,2,2,2,2,2,Mi momento favorito del fin de semana fue la c...,El seminario TAPS fue una experiencia transfor...
4,"0__Feeling_Immersed,_Connected_&_Seen",5,1,2,2,2,2,2,0,0,...,2,1,2,2,2,2,2,2,My favorite moment of the weekend was the oppo...,I would like to express my heartfelt gratitude...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,49__Navigating_Family_Relationships,2,3,2,2,2,2,2,0,1,...,2,1,2,2,2,2,2,2,One of my favorite moments from the TAPS semin...,The TAPS Seminar was a valuable experience for...
296,49__Learning_to_Process_Grief,3,1,2,2,2,2,2,0,0,...,2,1,2,2,2,2,2,2,"As a surviving child, the weekend seminar with...","El fin de semana pasado, mi momento favorito f..."
297,49__Moments_That_Matter,4,1,2,2,2,2,2,0,0,...,2,1,2,2,2,2,2,2,One of the most meaningful moments for me duri...,The TAPS Seminar experience exceeded my expect...
298,"49__Feeling_Immersed,_Connected_&_Seen",5,1,2,2,2,2,2,0,1,...,3,1,2,2,2,2,2,2,My favorite moment of the weekend was the grou...,The TAPS Seminar experience was truly transfor...


In [62]:
responses_df.to_csv('syntheticdata2.csv')